<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train1_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import pandas as pd

In [84]:

train_1 = pd.read_csv(r'/content/train_data1.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [85]:

train_1['Target'] = train_1['Target'].replace(['Others'],'Invalid')


In [86]:
train_1['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [87]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train1_Xtfidf = tfidf_vect.fit_transform(train_1['Sentence'])


In [89]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [90]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [91]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [92]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [93]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [94]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [95]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train1_Xtfidf,train_1['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6780973451327433
                  precision    recall  f1-score   support

        Analysis       0.50      0.39      0.44       126
      Conclusion       0.78      0.54      0.64        46
           Facts       0.74      0.91      0.82       485
         Invalid       0.51      0.42      0.46       132
           Issue       0.64      0.53      0.58        51
Rule/Law/Holding       0.52      0.20      0.29        64

        accuracy                           0.68       904
       macro avg       0.62      0.50      0.54       904
    weighted avg       0.65      0.68      0.65       904



In [96]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [97]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3]
[0.9908753392323401, 0.9804913972890953, 0.9971329576912025, 0.9998986125600647, 0.9979816266108709, 0.9970770119534687, 0.9968937603220123, 0.9980026680758075, 0.9913557364665678, 0.9986776585692952, 0.9980626510823426, 0.9947886917313501, 0.9969087275162024, 0.9850500637808548, 0.9800029488505851, 0.9902120380862394, 0.9952288179229508, 0.992854594371412, 0.9882653211336269, 0.9978302905988431, 0.9995141742452305, 0.9994539358247622, 0.9902361677581987, 0.9861325480471578, 0.9861540636316252, 0.9990050748145182, 0.9927947534134635, 0.9992500012898736, 0.9914847681015003, 0.9935344321359543, 0.9935334971974887, 0.9899992992123535, 0.999903412546073, 0.9950099534163821, 0.9800252886785016, 0.9879344239174176, 0.9959975170913764, 0.9925273674703293, 0.9989604770399253, 0.9994777967958088, 0.9992516449119875, 0.9970411222285332]
[0, 1, 4, 5, 11, 12, 17, 18, 21, 2

In [98]:
unlabel_1=unlabel_1.loc[[0, 1, 4, 5, 11, 12, 17, 18, 21, 23, 24, 29, 31, 33, 34, 35, 36, 37, 39, 40, 45, 48, 51, 52, 55, 56, 60, 62, 68, 69, 70, 72, 76, 78, 79, 84, 87, 89, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts


In [99]:
frame_1 = [train_1,unlabel_1]
train_1_1 = pd.concat(frame_1)
len(train_1_1)

3518

In [100]:
x_train_1 = tfidf_vect.transform(train_1_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train_1_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6789772727272727
                  precision    recall  f1-score   support

        Analysis       0.45      0.43      0.44        97
      Conclusion       0.78      0.57      0.66        37
           Facts       0.75      0.89      0.82       359
         Invalid       0.56      0.52      0.54       112
           Issue       0.72      0.50      0.59        42
Rule/Law/Holding       0.64      0.28      0.39        57

        accuracy                           0.68       704
       macro avg       0.65      0.53      0.57       704
    weighted avg       0.67      0.68      0.66       704



In [101]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9942790160997625, 0.9989806627807959, 0.9875356419732196, 0.9837869075237734, 0.9970024342805881, 0.9969658482707721, 0.9999465286602754, 0.9962035591914827, 0.9955626564011073, 0.9930663033474056, 0.9896175490783466, 0.995613301235158, 0.9999551250207184, 0.9993980846275048, 0.9997915048518421, 0.9875647780218351, 0.9992165864758218, 0.9842053372587841, 0.9957751137130098, 0.9801204334125695, 0.9994771213431248, 0.9992580102513319, 0.9998608773731016, 0.9989665953196241, 0.9996565782899723, 0.9948516609266145, 0.9969611260478841, 0.9892845713902667, 0.9970467660197635, 0.9814301540614058, 0.9922186828781407, 0.9991932550827388, 0.9976557685035939, 0.998933192099042, 0.9907827525673967]
[0, 2, 4, 6, 7, 8, 12, 21, 29, 31, 32, 36, 37, 39, 42, 48, 49, 54, 55, 58, 59, 63, 64, 69, 70, 71, 75, 79, 83, 85, 86, 88, 90, 91, 99]
35
35
35


In [102]:
unlabel_2 = unlabel_2.loc[[0,2, 4, 6, 7, 8, 12, 21, 29, 31,32, 34, 36, 37, 39, 42, 48, 49, 54, 55, 58, 59, 63, 64, 69, 70, 71, 75, 79, 83, 85, 86, 88, 90,91, 99 ],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [103]:
frame_2 = [train_1_1,unlabel_2]
train1_2 = pd.concat(frame_2)
len(train1_2)

3554

In [104]:
x_train_2 = tfidf_vect.transform(train1_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train1_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.628125
                  precision    recall  f1-score   support

        Analysis       0.41      0.37      0.39       142
      Conclusion       0.69      0.41      0.51        54
           Facts       0.69      0.90      0.78       467
         Invalid       0.51      0.41      0.45       158
           Issue       0.68      0.35      0.46        66
Rule/Law/Holding       0.75      0.29      0.42        73

        accuracy                           0.63       960
       macro avg       0.62      0.45      0.50       960
    weighted avg       0.62      0.63      0.60       960



In [105]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2]
[0.9977462642569433, 0.9992862474847651, 0.9943919067330006, 0.9958396721605416, 0.9864055159025074, 0.9997466223697687, 0.9996226840219401, 0.999755453461438, 0.9998866043904481, 0.9872110544137115, 0.9977758112818086, 0.9996935167183401, 0.9951696397629051, 0.9988142472479952, 0.9850703742988725, 0.9955935735581013, 0.999475251273419, 0.9909684089102547, 0.9996408769700763, 0.9811244786294886, 0.9965463479973903, 0.9963832957473366, 0.9953141341342243, 0.998456556949259, 0.9966888220389957, 0.9939575445477152, 0.9913142484510585, 0.9991775714428928, 0.9974923722162641, 0.9991974273119948, 0.9952884802303439, 0.9977282305786498, 0.9996778364153821, 0.996738506543136, 0.998109328173452, 0.9977421022987427, 0.981129226332051, 0.9990577971644518]
[0, 2, 3, 4, 6, 7, 8, 12, 20, 24, 26, 27, 29, 30, 33, 35, 36, 40, 43, 45, 46, 49, 51, 52, 54, 56, 57, 63, 64, 65, 66, 67, 68, 70, 

In [106]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 6,7, 8, 12, 20, 24, 26, 27, 29, 30, 33, 35, 36, 40, 43, 45, 46, 49, 51, 52, 54, 56, 57, 63, 64, 65, 66, 67, 68,70, 72,  75, 82, 89],:]
frame_3 = [train1_2,unlabel_3]
train1_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train1_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train1_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6675938803894298
                  precision    recall  f1-score   support

        Analysis       0.47      0.43      0.45       103
      Conclusion       0.72      0.55      0.63        38
           Facts       0.75      0.90      0.82       383
         Invalid       0.45      0.38      0.41       104
           Issue       0.70      0.52      0.60        44
Rule/Law/Holding       0.44      0.17      0.25        47

        accuracy                           0.67       719
       macro avg       0.59      0.49      0.52       719
    weighted avg       0.64      0.67      0.65       719



In [107]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9985477476035921, 0.9929631232011952, 0.999136394606327, 0.9999665100285274, 0.9995170703200124, 0.9835777607919683, 0.999636287328268, 0.9972208637345035, 0.9978531445905019, 0.9972627411865052, 0.9966841673509357, 0.9995016107926361, 0.9957713827557103, 0.9982547268435631, 0.994636305832651, 0.9997033196054446, 0.9986440188619858, 0.9990798855924242, 0.9949998143625843, 0.9998049308479395, 0.9884382658366709, 0.9976585157493897, 0.9845058767036067, 0.9959045449459994, 0.9984835899385123, 0.9894240980466176, 0.9886175682042504, 0.9807347729416224, 0.9874698049076227, 0.9926362324873608, 0.9928377332723498, 0.9928054987355495, 0.9976055225822965, 0.9991653778816675, 0.9999568548053728, 0.988975155118208, 0.9978396496950419, 0.9949824955173721, 0.9920184148320353, 0.9871393288565415, 0.9908001736625847, 0.9972732158356712, 0.9983568140080586]
[2, 6, 7, 10,

In [108]:
unlabel_4=unlabel_4.loc[[2, 6, 7, 10, 15, 17, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 39, 40, 42, 49, 50, 55, 56, 58, 60, 61, 63, 64, 67, 69, 70, 71, 75, 82, 85, 87, 89, 90, 91, 93, 97 ],:]

In [109]:
frame_4 = [train1_3,unlabel_4]
train1_4 = pd.concat(frame_4)
len(train1_4)
x_train_4 = tfidf_vect.transform(train1_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train1_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6832993890020367
                  precision    recall  f1-score   support

        Analysis       0.50      0.40      0.44       145
      Conclusion       0.70      0.46      0.55        50
           Facts       0.78      0.90      0.84       525
         Invalid       0.49      0.50      0.49       142
           Issue       0.54      0.51      0.53        49
Rule/Law/Holding       0.56      0.34      0.42        71

        accuracy                           0.68       982
       macro avg       0.60      0.52      0.55       982
    weighted avg       0.67      0.68      0.67       982



In [110]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2, 5, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 0]
[0.9989854115682758, 0.9832277524947313, 0.993887242052622, 0.9973056864638564, 0.9840435856706976, 0.9929807314802637, 0.996734439836905, 0.9983251054627292, 0.998614149783946, 0.9855538336880675, 0.9869107362290164, 0.9957663918235641, 0.9994831878416099, 0.9895012253449545, 0.9925773311439642, 0.9987389434476981, 0.9993617620042754, 0.9985547497167272, 0.993313222518663, 0.9980444464724952, 0.9947815734133256, 0.9913151309408634, 0.9959645400962472, 0.9921483793207824, 0.9958567329671443, 0.9882795188558511, 0.9999190382887572, 0.9990831245941997, 0.995742148664633, 0.999176715705388, 0.9983058275367237, 0.9891166050332248, 0.9842899085519851, 0.9886222460572208, 0.9866192026049491, 0.9899834406770172, 0.9901650601014129]
[3, 7, 8, 12, 18, 19, 22, 23, 26, 27, 29, 32, 36, 46, 48, 52, 53, 60, 63, 65, 73, 74, 75, 76, 77, 78, 82, 86, 93, 94, 95, 97, 100, 103, 104, 107, 110]
37

In [111]:
unlabel_5 = unlabel_5.loc[[3, 7, 8, 12, 18, 19, 22, 23, 26, 27, 29, 32, 36, 46, 48, 52, 53, 60, 63, 65, 73, 74, 75, 76, 77, 78, 82, 86, 93, 94, 95, 97, 100, 103, 104, 107, 110],:]

In [112]:
frame_5 = [train1_4,unlabel_5]
train1_5 = pd.concat(frame_5)
len(train1_5)
x_train_5 = tfidf_vect.transform(train1_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train1_5['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6884353741496598
                  precision    recall  f1-score   support

        Analysis       0.51      0.43      0.46       101
      Conclusion       0.77      0.50      0.61        34
           Facts       0.78      0.89      0.83       399
         Invalid       0.50      0.54      0.52       115
           Issue       0.60      0.34      0.44        35
Rule/Law/Holding       0.61      0.33      0.43        51

        accuracy                           0.69       735
       macro avg       0.63      0.51      0.55       735
    weighted avg       0.68      0.69      0.67       735



In [113]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6813819577735125
